# Pre-processing

- Select desired columns from each of the 11 datasets
- Preprocess and clean columns
- Append dataset to MRI (`processed.csv`)

**General process for each dataset**
- Sort each patient by most recent visit
- Remove duplicate visits keeping the most recent
- Merge dataset with aggregate dataset
- Check for null and unsubmitted values
- Impute values

All values have been preprocessed and imputed. Values are imputed by means of using the mean, median or value from most similar row.

All null values have been removed

In [1]:
import numpy as np
import pandas as pd

import os
import re

### Helpers

In [2]:
pattern = r'S_(\d*)'
def parse_rid(subject_id):
    # parse rid from subject_id
    rid = re.search(pattern, subject_id).group(1)
    return int(rid)

In [3]:
def find_similar_row(row, df, column=None):
    """
        Compare a row for a given dataframe against all other
        rows in the same dataframe.
        
        Calculate the norm of each row in a dataframe
        compared to the target row. Return the compared row
        with the smallest norm.
        
        NOTE: axis=1 is expected during df.apply usage which means:
            row.name === row_index
    """
    
    # remove target row from df and subtract target row values from remaining rows
    diff = df.drop(index=row.name)
    diff = diff - row.values
    
    # ensure all values are valid numbers before calculating norm
    diff = diff.fillna(value=0)
    norm_df = diff.apply(np.linalg.norm, axis=1)
    if column:
        try:
            return df.loc[norm_df.idxmin(), column]   
        except:
            raise KeyError(f"Invalid key: {column}")
    
    return df.loc[norm_df.idxmin(skipna=True), :]   

### Setup MRI with correct RID format

In [4]:
df = pd.read_csv('working/processed.csv')

In [5]:
df.shape

(617, 178)

In [6]:
df.head()

,SubjID,lh-Cerebellum-White-Matter,lh-Cerebellum-Cortex,lh-Thalamus-Proper,lh-Caudate,lh-Putamen,lh-Pallidum,3rd-Ventricle,4th-Ventricle,Brain-Stem,...,wm-rh-superiorfrontal,wm-rh-superiorparietal,wm-rh-superiortemporal,wm-rh-supramarginal,wm-rh-frontalpole,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,wm-lh-Unsegmented,wm-rh-Unsegmented
0,pvcrADNI_011_S_0021_PT_AV1451_2018021508521578...,1.122404,1.182973,1.817158,1.391494,1.799036,2.537900,0.965265,2.742845,1.074118,...,1.134314,1.018792,1.385129,1.172204,1.206521,1.469661,1.182140,1.214626,1.614727,1.587751
1,pvcrADNI_023_S_0031_PT_AV1451_2018042609172523...,1.021761,1.112750,1.303878,1.070731,1.652041,1.982748,1.250516,1.489708,1.178413,...,1.278840,1.128747,1.351128,1.211336,1.434194,1.470025,1.788683,1.324016,0.730924,0.814179
2,pvcrADNI_067_S_0056_PT_AV1451_2018022811275035...,0.933901,1.040130,1.540324,1.340539,1.969141,3.298904,0.830050,1.337372,0.983483,...,1.260969,1.316324,1.271024,1.296633,1.447026,1.191524,1.020604,1.384523,1.323594,1.330080
3,pvcrADNI_067_S_0059_PT_AV1451_2017122009132852...,1.003087,1.070947,1.503807,1.328975,1.878053,2.308336,0.919911,1.819666,0.984816,...,1.266842,1.245742,1.207279,1.322977,1.284196,1.209476,1.008124,1.253794,1.447955,1.407817
4,pvcrADNI_100_S_0069_PT_AV1451_2018040909483962...,1.042776,1.112264,1.267889,1.346736,2.233189,5.768378,1.248420,0.774037,1.073024,...,1.381038,1.418930,1.426585,1.400109,1.566343,1.650883,1.144346,1.416124,0.863427,0.926379


In [7]:
# rename subject ID column
df = df.rename(columns={'SubjID': 'RID'})
df['RID'] = df['RID'].apply(parse_rid)

In [8]:
df = df.drop_duplicates(subset=['RID'], keep='first')

### 1 - Alzheimer's disease sequencing project (ADSP)

Selected features:
- Roster ID
- Age
- Gender
- Phenotype Harmonization Consortium (PHC) - Composite Cognitive Scores
- Education length (years)
- Alzheimer's diagnosis

In [9]:
adsp = pd.read_csv('working/1_ADSP_PHC_COGN.csv')

In [10]:
# sort subjects by age (oldest first) 
# remove duplicate visits by keeping the first value (most recent visit)
cols_to_keep = ['RID', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'DX']
adsp = (adsp
           .sort_values('AGE', ascending=False)
           .drop_duplicates(subset=['RID'], keep='first')
          )[cols_to_keep]

In [11]:
adsp = adsp.sort_values('RID')

In [12]:
adsp.head()

,RID,AGE,PTGENDER,PTEDUCAT,PHC_MEM,PHC_EXF,PHC_LAN,DX
9,2,86.5489,1,16.0,-0.207,-0.703,0.195,2.0
13,3,83.3648,1,18.0,-1.559,-0.645,-0.756,3.0
16,4,70.8802,1,10.0,0.096,-0.417,0.624,2.0
21,5,76.7775,1,16.0,0.773,0.704,0.858,1.0
28,6,83.5209,2,13.0,-0.720,-0.234,0.364,2.0


In [13]:
df = df.merge(right=adsp, how='left', on='RID')

In [14]:
# replace na values with mean of each column
df['PHC_EXF'] = df['PHC_EXF'].fillna(df['PHC_EXF'].mean())
df['PHC_LAN'] = df['PHC_LAN'].fillna(df['PHC_LAN'].mean())

In [15]:
df = df.rename(columns={'DX': 'AD_LABEL'})

### 2 - Alzheimer's disease assessment scale (ADAS)

Selected features:
- Alzheimer's disease assessment scale total score

In [16]:
# ADNI 1 is stored in a separate dataset from ADNI 2, 3
adas_1 = pd.read_csv('working/2_1_ADAS_ADNI1.csv')
adas_2 = pd.read_csv('working/2_2 ADAS_ADNIGO23.csv')

In [17]:
adas_1 = (adas_1
          .sort_values(by='EXAMDATE', ascending=False)
          .drop_duplicates(subset=['RID'], keep='first')
          .sort_values(by='RID')
         )[['RID', 'CONMCXLA']]
adas_1 = adas_1.rename(columns={'CONMCXLA': 'ADAS_TOTAL'})

In [18]:
adas_2 = (adas_2
          .sort_values(by='USERDATE2', ascending=False)
          .drop_duplicates(subset=['RID'], keep='first')
          .sort_values(by='RID')
         )[['RID', 'TOTAL13']]
adas_2 = adas_2.rename(columns={'TOTAL13': 'ADAS_TOTAL'})

In [19]:
# concat ADAS dataframes
adas = pd.concat([adas_1, adas_2], ignore_index=True)

In [20]:
df = df.merge(right=adas, how='left', on='RID')
df['ADAS_TOTAL'] = df['ADAS_TOTAL'].fillna(df['ADAS_TOTAL'].median())

### 3 - Cognitive battery tests (CBB)

Selected features:
- Total correct
- Total incorrect

Engineered features:
- Cognitive battery score (%)

In [21]:
cbb = pd.read_csv('working/3_CBBRESULTS.csv')

In [22]:
cbb = pd.DataFrame(cbb.groupby(by='RID').agg(['mean']))[['TotalCorrect', 'TotalErrors']]

In [23]:
cbb = cbb.reset_index()

In [24]:
cbb.columns = ['RID', 'CORRECT', 'ERRORS']

In [25]:
cbb.head()

,RID,CORRECT,ERRORS
0,2,34.562500,12.625000
1,21,38.050000,7.550000
2,56,34.444444,19.555556
3,59,37.750000,8.250000
4,69,36.750000,9.166667


In [26]:
# calculate the percentage score from correct / errors
cbb['CBB_SCORE_%'] = 100 * (cbb['CORRECT'] / ( cbb['ERRORS'] + cbb['CORRECT']))

In [27]:
cbb = cbb.drop(columns=['CORRECT', 'ERRORS'])

In [28]:
df = df.merge(right=cbb, how='left', on='RID')

In [29]:
df = df.drop_duplicates(subset=['RID'])

In [30]:
# replace 101 missing CBB values with most similar row
to_replace = df['CBB_SCORE_%'].isnull()
df.loc[to_replace, 'CBB_SCORE_%'] = df[to_replace].apply(
    lambda row: find_similar_row(row, df, column='CBB_SCORE_%'),
    axis=1
)

In [31]:
# fill in last 10 values that could not be given a similar value
cbb_mean = df['CBB_SCORE_%'].mean()
df['CBB_SCORE_%'] = df['CBB_SCORE_%'].fillna(cbb_mean)

### 4 - Clinical Dimentia Rating (CDR)

Selected features:
- Clinical dimentia rating

In [32]:
cdr = pd.read_csv('working/4_CDR.csv')

In [33]:
cdr = (cdr
          .sort_values(by='EXAMDATE', ascending=False)
          .drop_duplicates(subset=['RID'], keep='first')
          .sort_values(by='RID')
         )[['RID', 'CDGLOBAL']]

In [34]:
df = df.merge(right=cdr[['RID', 'CDGLOBAL']], how='left', on='RID')

In [35]:
# drop patients with no label and no CDR score
drop = df[(df['CDGLOBAL'].isnull()) & (df['AD_LABEL'].isnull())].index
df = df.drop(drop)

In [36]:
# replace missing 11 CDR scores with AD_LABEL
df.loc[df['CDGLOBAL'].isnull(), 'CDGLOBAL'] = df[df['CDGLOBAL'].isnull()]['AD_LABEL']

In [37]:
# rename column
df = df.rename(columns={'CDGLOBAL': 'CDR'})

### 5 - Mini-mental state exam (MMSE)

Selected features:
- Mini-mental state score

In [38]:
mmse = pd.read_csv('working/5_MMSE_edited.csv')

In [39]:
mmse = (mmse
          .sort_values(by='EXAMDATE', ascending=False)
          .drop_duplicates(subset=['RID'], keep='first')
          .sort_values(by='RID')
         )[['RID', 'MMSCORE']]

In [40]:
df = df.merge(right=mmse[['RID', 'MMSCORE']], how='left', on='RID')

In [41]:
mmse = mmse.drop_duplicates(subset=['RID'])

In [42]:
df = df.rename(columns={'MMSCORE': 'MMSE'})

### 6 - Modified Hachinski Ischemia Scale (MODHACH)

Selected features:
- Modified Hachinski Ischemia Scale score

In [43]:
modhach = pd.read_csv('working/6_MODHACH.csv')

In [44]:
modhach = (modhach
           .sort_values(by='EXAMDATE', ascending=False)
           .drop_duplicates(subset=['RID'], keep='first')
           .sort_values(by='RID')
          )[['RID', 'HMSCORE']]

In [45]:
df = df.merge(right=modhach, how='left', on='RID')

In [46]:
df = df.rename(columns={'HMSCORE': 'MODHACH_SCORE'})

### 7 - Neuropsychiatric inventory (NPI)

Selected features:
- Neuropsychiatric inventory total

In [47]:
npi = pd.read_csv('working/7_NPI.csv')

In [48]:
npi = (npi
       .sort_values(by='EXAMDATE', ascending=False)
       .drop_duplicates(subset='RID', keep='first')
       .sort_values(by='RID')
      )[['RID', 'NPITOTAL']]

In [49]:
df = df.merge(right=npi, how='left', on='RID')

In [50]:
# fill remaining few values with median
df['NPITOTAL'] = df['NPITOTAL'].fillna(df['NPITOTAL'].median())

### 8 - Neuropsychological Battery (NEUROBAT)


Selected features:
- Logical memory immediate recall total
- Logical memory delayed recall total

In [51]:
neurobat = pd.read_csv('working/8_NEUROBAT_edited.csv')

In [52]:
neurobat = (neurobat
       .sort_values(by='EXAMDATE', ascending=False)
       .drop_duplicates(subset='RID', keep='first')
       .sort_values(by='RID')
      )[['RID', 'LIMMTOTAL', 'LDELTOTAL']]

In [53]:
neurobat = neurobat.rename(columns={
    'LIMMTOTAL': 'LOG_MEM_IMM_TOTAL',
    'LDELTOTAL': 'LOG_MEM_DEL_TOTAL'
})

In [54]:
df = df.merge(right=neurobat, how='left', on='RID')

In [55]:
to_replace = df['LOG_MEM_DEL_TOTAL'].isnull()
df.loc[to_replace, 'LOG_MEM_DEL_TOTAL'] = df[to_replace].apply(
    lambda row: find_similar_row(row, df, column='LOG_MEM_DEL_TOTAL'),
    axis=1
)
df['LOG_MEM_DEL_TOTAL'] = df['LOG_MEM_DEL_TOTAL'].fillna(df['LOG_MEM_DEL_TOTAL'].median())

In [56]:
to_replace = df['LOG_MEM_IMM_TOTAL'].isnull()
df.loc[to_replace, 'LOG_MEM_IMM_TOTAL'] = df[to_replace].apply(
    lambda row: find_similar_row(row, df, column='LOG_MEM_IMM_TOTAL'),
    axis=1
)
df['LOG_MEM_IMM_TOTAL'] = df['LOG_MEM_IMM_TOTAL'].fillna(df['LOG_MEM_IMM_TOTAL'].median())

### 9 - Neuropsych Summary Scores (NEUROPSYCH)

Selected features:
- Composite score for memory
- Composite score for executive functioning

In [57]:
neuropsych = pd.read_csv('working/9_UWNPSYCHSUM.csv')

In [58]:
neuropsych = (neuropsych
              .sort_values(by='EXAMDATE', ascending=False)
              .drop_duplicates(subset='RID', keep='first')
              .sort_values(by='RID')
             )[['RID', 'ADNI_MEM', 'ADNI_EF']]

In [59]:
df = df.merge(right=neuropsych, how='left', on='RID')

In [60]:
# replace 8 missing executive function scores
to_replace = df['ADNI_EF'].isnull()
df.loc[to_replace, 'ADNI_EF'] = df[to_replace].apply(
    lambda row: find_similar_row(row, df, column='ADNI_EF'),
    axis=1
)

In [61]:
df = df.rename(columns={
    'ADNI_MEM': 'COMP_MEM_SCORE', 
    'ADNI_EF': 'COMP_EXEC_FUNC_SCORE'
})

### Family history

Selected features:
- Mother dimentia
- Mother AD
- Father dimentia
- Father AD

In [62]:
fam_hist = pd.read_csv('working/11_FAMXHPAR.csv')

In [63]:
columns = ['RID', 'MOTHDEM', 'FATHDEM']
fam_hist = (fam_hist
              .sort_values(by='USERDATE2', ascending=False)
              .drop_duplicates(subset='RID', keep='first')
              .sort_values(by='RID')
             )[['RID', 'MOTHDEM', 'FATHDEM']]

In [64]:
df = df.merge(right=fam_hist, how='left', on='RID')

In [65]:
fill_remaining = [c for c in df.columns if c != 'AD_LABEL']
df[fill_remaining] = df[fill_remaining].fillna(0)
df['AD_LABEL'] = df['AD_LABEL'].fillna(1.0)

### Export

In [66]:
df.to_csv('working/combined.csv', index=False)